# Gather image metrics from metadata of images

## Create a dataframe with information about a subset of images
## Of Interest are:
- Samples per Pixel 
- Rows
- Columns
- Pixel Intensity Relationship

## I assume we want images with similar metrics to put them in an algorithm.
## If not, we can normalize using skimage.transform.resize. We would need to decide what to resize to

In [1]:
# Import the libraries we'll use below.
import os
import time
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

import pydicom

from collections import deque

In [2]:
# Make a dataframe of a subset of files that includes image metrics

# Create a deque to store the data
dicom_data = deque()

# Set the directory where your DICOM files are located
dicom_directory = r"F:\rsna-breast-cancer-detection\train_images"

# Set a counter for the number of DICOM files found
count = 0

# Start the timer
# We are doing a subset of file. Useful to have a timer to get an idea how long would take for all files
start_time = time.time()

# Recursively search for DICOM files in all subdirectories
for root, dirs, files in os.walk(dicom_directory):
    for filename in files:
        # Check if the file is a DICOM file
        if filename.endswith(".dcm"):
            # Use PyDicom to read the file
            ds = pydicom.dcmread(os.path.join(root, filename))
            # Extract the file name and image metrics
            file_name = filename
            samp_per_pix = ds.SamplesPerPixel
            n_rows_image = ds.Rows
            n_cols_image = ds.Columns
            # Add the data to the list
            dicom_data.append([file_name, samp_per_pix, n_rows_image, n_cols_image])
            # Increment the counter
            count += 1
            if count % 200 == 0:
                print(f"Processed {count} DICOM files...")
            # Check if the count has reached 5000 and exit the loop if it has
            if count == 5000:
                break
    # Check if the count has reached 5000 and exit the loop if it has
    if count >= 5000:
        break

# Stop the timer
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Processed 5000 DICOM files in {elapsed_time:.2f} seconds")
        
# Create a Pandas DataFrame from the list
df = pd.DataFrame(dicom_data, columns=['file_name', 'samp_per_pix', 'n_rows_image', 'n_cols_image'])

Processed 200 DICOM files...
Processed 400 DICOM files...
Processed 600 DICOM files...
Processed 800 DICOM files...
Processed 1000 DICOM files...
Processed 1200 DICOM files...
Processed 1400 DICOM files...
Processed 1600 DICOM files...
Processed 1800 DICOM files...
Processed 2000 DICOM files...
Processed 2200 DICOM files...
Processed 2400 DICOM files...
Processed 2600 DICOM files...
Processed 2800 DICOM files...
Processed 3000 DICOM files...
Processed 3200 DICOM files...
Processed 3400 DICOM files...
Processed 3600 DICOM files...
Processed 3800 DICOM files...
Processed 4000 DICOM files...
Processed 4200 DICOM files...
Processed 4400 DICOM files...
Processed 4600 DICOM files...
Processed 4800 DICOM files...
Processed 5000 DICOM files...
Processed 5000 DICOM files in 476.40 seconds


In [3]:
df.head()

,file_name,samp_per_pix,n_rows_image,n_cols_image
0,1459541791.dcm,1,5355,4915
1,1864590858.dcm,1,5355,4915
2,1874946579.dcm,1,5355,4915
3,462822612.dcm,1,5355,4915
4,1031443799.dcm,1,2776,2082


In [4]:
# Output dataframe to file for future use
output_file = "ImageMetrics_Subset.csv"
df.to_csv(output_file, index=False)

In [6]:
# upload dataframe
df = pd.read_csv(r"C:\Users\mandy\Documents\MIDS\W207\W207MammogramProject\ImageMetrics_Subset.csv")

In [12]:
# Get summary of value counts for each metric
#samples per pixel
print( df['samp_per_pix'].value_counts(),'\n')
print( df['n_rows_image'].value_counts(),'\n')
print( df['n_cols_image'].value_counts())

1    5000
Name: samp_per_pix, dtype: int64 

4096    2249
5355     849
3328     763
2776     718
2294     179
3062     141
4740      54
5928      40
2850       5
1236       2
Name: n_rows_image, dtype: int64 

3328    2249
4915     849
2560     763
2082     718
1914     179
2394     146
3540      54
4728      40
1022       2
Name: n_cols_image, dtype: int64
